# <center> PySpark Streaming </center>

In [1]:
!module list

Currently Loaded Modulefiles:
  1) anaconda3/4.2.0   2) matlab/2015a      3) zeromq/4.1.5


In [ ]:
!mkdir ~/software
!cd ~/software; wget https://archive.apache.org/dist/spark/spark-2.1.0/spark-2.1.0-bin-hadoop2.7.tgz

In [ ]:
!cd ~/software; tar -xzf spark-2.1.0-bin-hadoop2.7.tgz

In [2]:
import os
import subprocess

def generate_compute_file(node_file=os.environ['PBS_NODEFILE'], 
                         slaves_file='/home/lngo/software/spark-2.1.0-bin-hadoop2.7/conf/slaves'):
    
    node_list = subprocess.check_output(['cat', node_file]).decode("utf-8").strip().split('\n')
    node_list = sorted(list(set(node_list)))

    print("Executors: ") 
    with open(slaves_file, 'w') as slave_file_fp:
        for node in node_list:
            slave_file_fp.write('{}\n'.format(node))
            print(node)
generate_compute_file()

Executors: 
node0052.palmetto.clemson.edu
node0121.palmetto.clemson.edu
node0139.palmetto.clemson.edu
node1774.palmetto.clemson.edu


In [3]:
!more ~/software/spark-2.1.0-bin-hadoop2.7/conf/slaves

node0052.palmetto.clemson.edu
node0121.palmetto.clemson.edu
node0139.palmetto.clemson.edu
node1774.palmetto.clemson.edu


In [4]:
!sh ~/software/spark-2.1.0-bin-hadoop2.7/sbin/stop-all.sh
!echo '------------------------------------------------------'
!sh ~/software/spark-2.1.0-bin-hadoop2.7/sbin/start-all.sh

node1774.palmetto.clemson.edu: stopping org.apache.spark.deploy.worker.Worker
node0052.palmetto.clemson.edu: stopping org.apache.spark.deploy.worker.Worker
node0121.palmetto.clemson.edu: stopping org.apache.spark.deploy.worker.Worker
node0139.palmetto.clemson.edu: stopping org.apache.spark.deploy.worker.Worker
stopping org.apache.spark.deploy.master.Master
------------------------------------------------------
starting org.apache.spark.deploy.master.Master, logging to /home/lngo/software/spark-2.1.0-bin-hadoop2.7/logs/spark-lngo-org.apache.spark.deploy.master.Master-1-node1774.palmetto.clemson.edu.out
node1774.palmetto.clemson.edu: starting org.apache.spark.deploy.worker.Worker, logging to /home/lngo/software/spark-2.1.0-bin-hadoop2.7/logs/spark-lngo-org.apache.spark.deploy.worker.Worker-1-node1774.palmetto.clemson.edu.out
node0052.palmetto.clemson.edu: starting org.apache.spark.deploy.worker.Worker, logging to /home/lngo/software/spark-2.1.0-bin-hadoop2.7/logs/spark-lngo-org.apache.sp

In [5]:
import sys
import os
import json

sys.path.insert(0, '/home/lngo/software/spark-2.1.0-bin-hadoop2.7/python')
sys.path.insert(0, '/home/lngo/software/spark-2.1.0-bin-hadoop2.7/python/lib/py4j-0.10.4-src.zip')

os.environ["SPARK_HOME"] = "/home/lngo/software/spark-2.1.0-bin-hadoop2.7/"
os.environ['PYSPARK_PYTHON'] = '/software/anaconda/4.2.0/bin/python'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.0 pyspark-shell'

import pyspark
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

conf = pyspark.SparkConf()
conf.setMaster("spark://node1774:7077")
conf.set("spark.driver.memory","8g")
conf.set("spark.executor.memory","50g")
conf.set("spark.num.executors","4")
conf.set("spark.executor.cores","12")

sc = pyspark.SparkContext(conf=conf, appName='PythonStreaming')
ssc = StreamingContext(sc, 2)

In [6]:
sc

In [7]:
ssc

In [8]:
brokers = 'node1774.palmetto.clemson.edu:9092'
topic = 'test'

def parseTweet(x):
    textContent = 'NoContent'
    try:
        jsonData = json.loads(json.loads(x))
        textContent = jsonData['text']
    except TypeError:
        textContent = 'TypeError'
    return textContent

kvs = KafkaUtils.createDirectStream(ssc, [topic],{'metadata.broker.list': brokers})
lines = kvs.map(lambda x: x[1])
counts = lines.map(parseTweet).flatMap(lambda line: line.split(' ')) \
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda a, b: a+b)

counts.pprint()
ssc.start()
ssc.awaitTermination()

-------------------------------------------
Time: 2017-11-27 13:07:48
-------------------------------------------

-------------------------------------------
Time: 2017-11-27 13:07:50
-------------------------------------------
(u'we', 1)
(u'UGA,', 1)
(u'@JBarnesRTR', 1)
(u'I', 1)
(u'Oklahoma', 1)
(u'mean', 1)
(u'looked', 1)
(u'all', 1)
(u'at', 1)
(u'https://t.co/8yQRTR5Odc', 1)
...

-------------------------------------------
Time: 2017-11-27 13:07:52
-------------------------------------------

-------------------------------------------
Time: 2017-11-27 13:07:54
-------------------------------------------

-------------------------------------------
Time: 2017-11-27 13:07:56
-------------------------------------------

-------------------------------------------
Time: 2017-11-27 13:07:58
-------------------------------------------

-------------------------------------------
Time: 2017-11-27 13:08:00
-------------------------------------------
(u'LSU.', 1)
(u'Clemson', 1)
(u'to', 1

-------------------------------------------
Time: 2017-11-27 13:09:14
-------------------------------------------

-------------------------------------------
Time: 2017-11-27 13:09:16
-------------------------------------------

-------------------------------------------
Time: 2017-11-27 13:09:18
-------------------------------------------

-------------------------------------------
Time: 2017-11-27 13:09:20
-------------------------------------------
(u'', 1)
(u'will', 1)
(u'Like', 1)
(u'like', 1)
(u'https://t.co/j4hkivn071', 1)
(u'take', 1)
(u'Miami', 1)
(u'I', 2)
(u'is', 1)
(u'top-15', 1)
...

-------------------------------------------
Time: 2017-11-27 13:09:22
-------------------------------------------
(u'defense', 1)
(u'for', 1)
(u'Bentley', 1)
(u'Clemson', 1)
(u'life', 1)
(u'made', 1)
(u'football:', 1)
(u'Carolina', 1)
(u'Jake', 1)
(u'https://t.co/uSHDa8jdPU', 1)
...

-------------------------------------------
Time: 2017-11-27 13:09:24
--------------------------------------

-------------------------------------------
Time: 2017-11-27 13:10:48
-------------------------------------------



KeyboardInterrupt: 

To stop the Spark job, call `sc.stop()`

In [9]:
ssc.stop()
sc.stop()

-------------------------------------------
Time: 2017-11-27 13:10:52
-------------------------------------------

